In [1]:
import pandas as pd
import numpy as np
import netCDF4
from netCDF4 import Dataset
import os
from pyproj import Proj
from datetime import datetime
import datetime
import sys
import time
from netCDF4 import num2date, date2num
import h5py
import IPython
import time
import timeit
from IPython.display import display, HTML
import os
import glob
#from pyhdf.SD import SD, SDC
import sys

UL = (-71.163736,   42.458042 )
BR = (-71.006377,   42.278300)
spatial_interval = 20  # meter
time_interval = 60     # miniute
mid_point = ((UL[0] + BR[0])/2, (UL[1]+ BR[1])/2)
UTM_zone = int((mid_point[0])/6) + 30
if mid_point[0] > 0:   # needs to be verified.
    UTM_zone +=1
# print(int(-71.1129200974982/6))
# print((UTM_zone))
#print((tweets))
start_date = datetime.datetime(2017, 1, 1)
end_date = datetime.datetime(2018, 4, 5) + datetime.timedelta(days=1)
delta_time = end_date - start_date

layer_cnt = int(delta_time.total_seconds() /60/time_interval) + 1

print('layer_cnt: ', layer_cnt)

print("mid_point: ", mid_point)


sphere = 'north'
if  mid_point[1] < 0:
    sphere = 'south'

myProj = Proj("+proj=utm +zone={}, +{}+ellps=WGS84 +datum=WGS84 +units=m +no_defs".format(UTM_zone, sphere))
UTMx, UTMy = myProj(mid_point[0], mid_point[1])
print(UTMx, UTMy )
lon, lat = myProj(UTMx + spatial_interval, UTMy + spatial_interval, inverse=True)
resolu_x = lon - mid_point[0]
resolu_y = lat - mid_point[1]
cols = int((BR[0] - UL[0])/resolu_x)+1
rows = int((UL[1] - BR[1])/resolu_y)+1
print(rows," ", cols)

def ReadTweetsInRectangle(csv_path,UL, BR):
    tweets = pd.read_csv(csv_path, encoding="utf-8-sig")
    tweets_clip = tweets[
        (tweets.longitude > UL[0]) & (tweets.longitude < BR[0]) & (tweets.latitude < UL[1]) & (tweets.latitude > BR[1])]
    print('len(tweets_clip): ', len(tweets_clip))
    print('ReadTweetsInRectangle finished. ')
    return tweets_clip


csv_path = 'D:\\Buston\\Boston2017_18.csv'
%time twts = ReadTweetsInRectangle(csv_path, UL, BR)

twts['userVerified']= twts['userVerified'].astype(bool)

twts['date'] = pd.to_datetime(twts['date'])

twts = twts.set_index('date')
twts = twts.sort_index()
twts['date'] = twts.index




day_twts = twts.loc['20170101']

def UserCntInBin(df,res_x, res_y, ori_x, ori_y):
    try:
        #print(df.loc[5])
        row_cnt = len(df)
        print('row_cnt: ', row_cnt)
        i = 0

        idx_lon = df.columns.get_loc('longitude')
        idx_lat = df.columns.get_loc('latitude')
        idx_time = df.columns.get_loc('date')
        idx_userID = df.columns.get_loc('userID')

        print("Start to get the user number in each bin...")
        df['row'] = 0
        df['col'] = 0
        df['layer'] = 0
        df['xyz'] = 'abc'

        idx_row = df.columns.get_loc('row')
        idx_col = df.columns.get_loc('col')
        idx_layer = df.columns.get_loc('layer')
        idx_xyz = df.columns.get_loc('xyz')

        while i < row_cnt:
            #print(df.iloc[i])
            #print(df.iat[i, idx_lon])
            #print(df.iat[i, idx_lat])

            row, col = GetRowCol(res_x, res_y, df.iat[i, idx_lon], df.iat[i, idx_lat], ori_x, ori_y)
            post_date = datetime.datetime.strptime(df.iat[i, idx_time],  '%Y-%m-%d %H:%M:%S')
            time_past = ( post_date - start_date)
            layer = int(time_past.total_seconds()/60/time_interval)
            #print(post_date)

            #cube[col, row, layer] +=1
            df.iat[i, idx_row] = row
            df.iat[i, idx_col] = col
            df.iat[i, idx_layer] = layer


            if i % 100000 == 0:
                print('df.iat[i, idx_layer]: ', df.iat[i, idx_layer])
            i = i + 1
        df['xyz'] = df['row'].astype(str) + "," + df["col"].astype(str) + ',' + df["layer"].astype(str)
        userCnts = df.groupby('xyz').size().reset_index(name='counts')
        print("User number in each bin was got. ")
        #print('df[idx_xyz]: ', df[df.xyz != "0,0,0"] )
        #print('userCnts: ', userCnts.columns)
        print('userCnts.counts.sum(): ', userCnts.counts.sum())
        print('len(userCnts): ', len(userCnts))
        #print('userCnts: ', userCnts[userCnts.counts > 1])

        # Fill the cube
        row_cnt = len(userCnts)
#         i = 0
#         while i < row_cnt:
#             row, col, layer = userCnts.iat[i, 0].split(",")
#             #print('row, col, layer: ', row, col, layer)
#             cube[int(col), int(row), int(layer)] += 1

#             i += 1
#         print(cube)
        #cube.tofile('cube_userCnt.bin')
        return userCnts
        print("Finished! ")
    except Exception as e:
        print("Get user number  fail error: ", repr(e))
        #print(df.iloc[i])

        
        
def SaveHDF5(cube, path_name):
    print("Start to save HDF5...")
    with h5py.File(path_name, 'w') as outfile:
        dset = outfile.create_dataset('S_T_cube', data=cube, chunks=True, compression="gzip")
        dset.attrs['sum_user'] = 'The sum of users.'
    print("Save HDF5 finished.")
    
    

C:\Users\GPU\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


layer_cnt:  11041
mid_point:  (-71.08505650000001, 42.368171000000004)
328315.99825595925 4692760.795869267
975   665


C:\Users\GPU\Anaconda3\lib\site-packages\IPython\core\magic.py:187: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


len(tweets_clip):  848017
ReadTweetsInRectangle finished. 
Wall time: 14 s


In [29]:
#def UserCntInBin(cube, df,res_x, res_y, ori_x, ori_y):

        
#print(df.loc['20170101'])

#UserCntInBin(df.loc['20170101'], resolu_x, resolu_y, UL[0], UL[1])


def ReadHDF5(path_name):
    f = h5py.File(path_name, 'r')
    print(f)
    print('sys.getsizeof(f): ', sys.getsizeof(f))
    Tweets_cnt = f['Data Fields/Tweets_Cnt'][:,:,23:24].sum()
    User_Cnt = f['Data Fields/User_Cnt'][:,:,23:24].sum()
    #print('sys.getsizeof(S_T_cube): ', sys.getsizeof(S_T_cube))
    print('Tweets_cnt: ', Tweets_cnt)
    print(User_Cnt)
    print('sys.getsizeof(f): ', sys.getsizeof(f))
    f.close()
#     print('len(f.keys()): ', len(f.keys()))
#     print('f.keys(): ', f.keys())
#     #S_T_cube = f['S_T_cube'][:,:,:]
#     #print(f['S_T_cube'].max())
#     #cu = f['S_T_cube'][:, :, :]#.astype('uint8')
#     #quarts = []
#     for i in range(f['S_T_cube'].shape[2]):
#         #print(f['S_T_cube'][:, :, i].sum())
#         quarts.append(f['S_T_cube'][3, 3, i])
#         if i % 10 == 0:
#             print(i)
#             #print(quarts[i])
#     #print(cu.sum())
    #print(f)
    

HDF_file = 'D:\\Buston\\HDF5_20170101.hdf'  
#HDF_file = 'E:\Data\HDF5_20170101.hdf'
#f = h5py.File(HDF_file, 'r')

#f = SD(HDF_file, SDC.READ)
#print(f.info())
%time ReadHDF5(HDF_file)

<HDF5 file "HDF5_20170101.hdf" (mode r)>
sys.getsizeof(f):  56
Tweets_cnt:  48
33
sys.getsizeof(f):  56
Wall time: 4.01 ms


In [211]:
df['username']= df['username'].astype(str)

 
for col in df.columns:
    #print(col)
    if df[col].dtype  == 'object':
        print(col)
        print(df[col].dtype)
        df[col] = df[col].astype('str')
        
        print(df[col].dtype)
        
print(df.dtypes)

# store = pd.HDFStore('D:\\Buston\\Boston2017_18_compres9.h5',complevel=9, complib='bzip2')
# store['Tweets'] = df
# store.close()



username
object
object
date
object
object
message
object
object
geoType
object
object
place
object
object
source
object
object
userMentions
object
object
urls
object
object
hashtags
object
object
userLocation
object
object
joinDay
object
object
language
object
object
userVerified
object
object
userDescription
object
object
userUrl
object
object
placeType
object
object
bboxType
object
object
placeId
object
object
geo
object
object
tweetID              int64
userID               int64
username            object
date                object
message             object
geoType             object
longitude          float64
latitude           float64
place               object
placeBboxWest      float64
placeBboxEast      float64
placeBboxSouth     float64
placeBboxNorth     float64
source              object
userMentions        object
urls                object
hashtags            object
retweetCount       float64
userLocation        object
followersCount     float64
friendsCount       float64

In [32]:
df['username'].astype(str)

0          tmj_bos_health
1             tweetsbyNAS
2              kviksilver
3         JohnHancockJobs
4            tmj_bos_itpm
5            speakotosico
6          tmj_bos_retail
7            echobostonRE
8            ManulifeJobs
9            tmj_bos_mgmt
10                MaidenV
11            HydeCareers
12        JobsAtContinuum
13        christinedtrant
14              DTRModern
15         tmj_bos_health
16         tmj_bos_cstsrv
17            tmj_bos_adv
18               jazzyrae
19             heidi_robb
20          tmj_bos_pharm
21        tmj_bos_nursing
22                MaidenV
23                zionano
24         tmj_BOS_intern
25             LazoAthena
26         tmj_bos_health
27            dniceboston
28            tmj_bos_eng
29         LloydLegacy617
               ...       
954207     BostonProperty
954208      CVSHealthJobs
954209           bobong61
954210        _Matthew_ME
954211    DJF_Investments
954212     ReginaldoBRUSA
954213      ariatrattoria
954214      

In [ ]:
from multiprocessing import Pool
import time


def task(msg):
    print ('hello, ', msg)
    time.sleep(1)
    return 'msg: '+ msg

#msgs = ['a', 'b', 'c', 'd', 'e']

if __name__ == '__main__':
    pool = Pool(processes=4)

    results = []
    msgs = [x for x in range(10)]
    results = pool.map(task, msgs)

    pool.close()
    pool.join()

    print('processes done, result:')

    for x in results:
        print(x)

In [98]:
import requests
import selenium
from selenium import webdriver
import lxml
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options


WINDOW_SIZE = "100,100"
webdriver_path = 'D:\\Thesis\\Tweets_image_download\\Webdriver\\chromedriver.exe'
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
Loading_time = 3
driver = webdriver.Chrome(executable_path=webdriver_path, chrome_options=chrome_options)
driver.set_page_load_timeout(Loading_time)
    
url = 'https://t.co/fIs7rVZH78'
%time session = requests.Session()  # so connections are recycled
#%time resp = session.head(url, allow_redirects=True)
url = 'https://t.co/fIs7rVZH78'
%time resp = session.head('https://www.instagram.com/p/BPTbab0lGF2/', allow_redirects=True)
print(resp.status_code )

#%time driver.get(url)
%time driver.get('https://www.insram.com/pDckBPog/')
print(driver.getCUrrentUrl())

Wall time: 0 ns
Wall time: 156 ms
404


TimeoutException: Message: timeout
  (Session info: headless chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.16299 x86_64)


AttributeError: 'WebDriver' object has no attribute 'getCUrrentUrl'

In [99]:
# regular expression test
import re
csv_path = 'D:\\Quadtree\\Place\\2017-01place1.csv'
twts = pd.read_csv(csv_path, error_bad_lines=False)
#print(twts['message'])
i = 0

for j in range(10):
    t = twts['message'][j]
    if len(str(t)) > 3:
        #print(t)
        m = re.findall('https:\/\/t.co\/[a-zA-Z0-9]+', t)
        if m:
            #print(m)
            i +=1
        #if len(m.string) > 0:
print(i)
         #   print(m.string)

KeyError: 'message'

tweetID         int64
 latitude     float64
 longitude    float64
 message       object
 source        object
 postdate      object
 userUrl       object
dtype: object
815576685240352768


In [123]:
csv_path = 'D:\\Quadtree\\Place\\2017-01place1.csv'
twts = pd.read_csv(csv_path, error_bad_lines=False)
twts = twts.fillna('')
i = 0
def GetTwtsUrl(twts):
    return ';'.join(re.findall('https:\/\/t.co\/[a-zA-Z0-9]+', twts['message'])) 

#%time month_twts['quadtree'] = month_twts.apply(GetQuadtree, axis=1)

%time twts['urls'] = twts.apply(GetTwtsUrl, axis=1) 

for j in range(5):
    print(twts['new_url'][j])

Wall time: 1min 39s


KeyError: 'new_url'

In [122]:
#%time twts['urls'] = twts.apply(GetTwtsUrl, axis=1) 
print(twts.dtypes)
print(twts['tweetID'])

tweetID         int64
 latitude     float64
 longitude    float64
 message       object
 source        object
 postdate      object
 userUrl       object
dtype: object
0          815576685240352768
1          815576685533863936
2          815576685508775936
3          815576685672333312
4          815576686414667776
5          815576686364336129
6          815576686469074944
7          815576686431531009
8          815576685349367809
9          815576687027036161
10         815576687458926592
11         815576686230208513
12         815576687589081088
13         815576687350050817
14         815576687895347200
15         815576689463926784
16         815576689602392064
17         815576690592194560
18         815576690667687936
19         815576690332233728
20         815576689828839426
21         815576691221237760
22         815576691598733312
23         815576691359838208
24         815576691779182592
25         815576690596450305
26         815576692437549056
27         81557669236

In [72]:
#twts = twts.fillna('')

for j in range(900, 1000):
    if (twts['new_url'][j]+ ';') != twts['urls'][j]:
        print("j: ",j,  twts['new_url'][j] + " <> " + str(twts['urls'][j]))
 

j:  900 https://t.co/IToDLNfsuq <> 
j:  901 https://t.co/yVoLa7Sh7O <> 
j:  904  <> 
j:  905  <> 
j:  907 https://t.co/i5ndts9sI1 <> 
j:  908 https://t.co/7sl <> 
j:  910 https://t.co/eKXlZcTQqd <> 
j:  912 https://t.co/V <> 
j:  913  <> 
j:  914 https://t.co/UZclCM0X8y <> 
j:  915 https://t.co/LRgmdBtMry <> 
j:  916 https://t.co/DdaK7bZ5m6 <> 
j:  917 https://t.co/GrtmU9e7OV <> 
j:  918 https://t.co/3at83LtJtR <> 
j:  920 https://t.co/oBx6IGqJzw <> 
j:  922 https://t.co/N0NNQU9GQh <> 
j:  923 https://t.co/H2N2LKqBuD <> 
j:  924  <> 
j:  925  <> 
j:  926 https://t.co/rDdnkNVoGt <> 
j:  927 https://t.co/jrslbUu0gf;https://t.co/phqKM4e0nx <> https://t.co/jrslbUu0gf;
j:  928  <> 
j:  929  <> 
j:  930  <> 
j:  931  <> 
j:  932  <> 
j:  933  <> 
j:  934  <> 
j:  935  <> 
j:  936  <> 
j:  938 https://t.co/WpU1zV7Tkr <> 
j:  939  <> 
j:  940  <> 
j:  941  <> 
j:  943 https://t.co/2aH6tp9qtT <> 
j:  944 https://t.co/ZWwzfFyYg1 <> 
j:  946  <> 
j:  949  <> 
j:  950  <> 
j:  951  <> 
j:  952  <>

In [1]:
import os
import csv
import pandas as pd
import glob

worldfile_ext = '.PGw'
train_csv = r'H:\RS\Lexington\train.csv'

img_path = r'H:\RS\Lexington\PNG'
all_files = glob.glob(img_path + '\*' + worldfile_ext)


f = open(all_files[0], 'r')
x = f.readlines()
f.close()


train_df = pd.read_csv(train_csv)
train_df['Id'] = train_df['Id'].astype(str)
# (x[4])
print(train_df.dtypes)
prefix = "D"
extend = 20
print(os.path.dirname(train_csv))
w = open(os.path.join(os.path.dirname(train_csv), "boxfile.txt") ,'w')
for file in all_files:
    image_id = os.path.basename(file)
    #Id = (image_id.split('.')[0].replace('D', ''))
    Id = (image_id.split('.')[0])
    print('file: ', file)
    f = open(file, 'r')
    x = f.readlines()
    f.close()
    
    df = train_df[train_df['Id'] == Id]
    
    df['POINT_X'] = df['POINT_X'] - float(x[4])
    df['POINT_Y'] = -df['POINT_Y'] + float(x[5])
    df['class'] = '0 '
    df['box'] = (df['POINT_X'].round(decimals=0).astype(int) - extend).astype(str) + ',' + \
            (df['POINT_Y'].round(decimals=0).astype(int) - extend).astype(str) + ',' + \
            (df['POINT_X'].round(decimals=0).astype(int) + extend).astype(str) + ',' + \
            (df['POINT_Y'].round(decimals=0).astype(int) + extend).astype(str) + ',' + \
            df['class'] 
    if len(df) > 0:        
        print('len(df): ', len(df))
        re  = df.groupby('Id')['box'].apply(lambda x: x.sum())
        w.write(img_path + '\\'+ Id + ".PNG ")
        w.write(str(re[0]) + '\n')
        #print(type(re))
    #re['Id'] = img_path + '\\'+ Id + ".JPG"
    #print(re)
    #re = re[['Id', 'box']]
    
w.close()
#df['box']
#df.groupby('Id')['box'].apply(lambda x: "{%s}" % ' '.join(x))

    

POINT_X    float64
POINT_Y    float64
Id          object
dtype: object
H:\RS\Lexington
file:  H:\RS\Lexington\PNG\0.PGw
len(df):  3
file:  H:\RS\Lexington\PNG\1.PGw


C:\Users\GPU\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\GPU\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\GPU\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

len(df):  4
file:  H:\RS\Lexington\PNG\10.PGw
len(df):  5
file:  H:\RS\Lexington\PNG\100.PGw
file:  H:\RS\Lexington\PNG\1000.PGw
len(df):  4
file:  H:\RS\Lexington\PNG\1001.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\1002.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\1003.PGw
len(df):  8
file:  H:\RS\Lexington\PNG\1004.PGw
len(df):  6
file:  H:\RS\Lexington\PNG\1005.PGw
len(df):  5
file:  H:\RS\Lexington\PNG\1006.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\1007.PGw
len(df):  5
file:  H:\RS\Lexington\PNG\1008.PGw
len(df):  15
file:  H:\RS\Lexington\PNG\1009.PGw
len(df):  8
file:  H:\RS\Lexington\PNG\101.PGw
file:  H:\RS\Lexington\PNG\1010.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\1011.PGw
len(df):  4
file:  H:\RS\Lexington\PNG\1012.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\1013.PGw
len(df):  13
file:  H:\RS\Lexington\PNG\1014.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\1015.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\1016.PGw
len(df):  3
file:  H:\RS\Lexington\PNG\1017.PGw
len(df):  4
file: 

len(df):  1
file:  H:\RS\Lexington\PNG\1157.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\1158.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\1159.PGw
len(df):  3
file:  H:\RS\Lexington\PNG\116.PGw
file:  H:\RS\Lexington\PNG\1160.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\1161.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\1162.PGw
len(df):  10
file:  H:\RS\Lexington\PNG\1163.PGw
len(df):  3
file:  H:\RS\Lexington\PNG\1164.PGw
len(df):  3
file:  H:\RS\Lexington\PNG\1165.PGw
len(df):  3
file:  H:\RS\Lexington\PNG\1166.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\1167.PGw
len(df):  13
file:  H:\RS\Lexington\PNG\1168.PGw
len(df):  3
file:  H:\RS\Lexington\PNG\1169.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\117.PGw
file:  H:\RS\Lexington\PNG\1170.PGw
len(df):  5
file:  H:\RS\Lexington\PNG\1171.PGw
len(df):  7
file:  H:\RS\Lexington\PNG\1172.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\1173.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\1174.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\1175.PGw
len(df):  1
file

len(df):  6
file:  H:\RS\Lexington\PNG\1324.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\1325.PGw
len(df):  4
file:  H:\RS\Lexington\PNG\1326.PGw
len(df):  7
file:  H:\RS\Lexington\PNG\1327.PGw
len(df):  5
file:  H:\RS\Lexington\PNG\1328.PGw
len(df):  9
file:  H:\RS\Lexington\PNG\1329.PGw
len(df):  5
file:  H:\RS\Lexington\PNG\133.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\1330.PGw
len(df):  3
file:  H:\RS\Lexington\PNG\1331.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\1332.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\1333.PGw
len(df):  6
file:  H:\RS\Lexington\PNG\1334.PGw
len(df):  9
file:  H:\RS\Lexington\PNG\1335.PGw
len(df):  5
file:  H:\RS\Lexington\PNG\1336.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\1337.PGw
len(df):  6
file:  H:\RS\Lexington\PNG\1338.PGw
len(df):  3
file:  H:\RS\Lexington\PNG\1339.PGw
len(df):  9
file:  H:\RS\Lexington\PNG\134.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\1340.PGw
len(df):  13
file:  H:\RS\Lexington\PNG\1341.PGw
file:  H:\RS\Lexington\PNG\1342.PGw
file:

...

len(df):  14
file:  H:\RS\Lexington\PNG\8295.PGw
len(df):  48
file:  H:\RS\Lexington\PNG\8296.PGw
len(df):  24
file:  H:\RS\Lexington\PNG\8297.PGw
len(df):  61
file:  H:\RS\Lexington\PNG\8298.PGw
len(df):  27
file:  H:\RS\Lexington\PNG\8299.PGw
len(df):  71
file:  H:\RS\Lexington\PNG\83.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\830.PGw
len(df):  6
file:  H:\RS\Lexington\PNG\8300.PGw
len(df):  41
file:  H:\RS\Lexington\PNG\8301.PGw
len(df):  12
file:  H:\RS\Lexington\PNG\8302.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\8303.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\8304.PGw
file:  H:\RS\Lexington\PNG\8305.PGw
file:  H:\RS\Lexington\PNG\8306.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\8307.PGw
len(df):  3
file:  H:\RS\Lexington\PNG\8308.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\8309.PGw
len(df):  18
file:  H:\RS\Lexington\PNG\831.PGw
len(df):  21
file:  H:\RS\Lexington\PNG\8310.PGw
len(df):  3
file:  H:\RS\Lexington\PNG\8311.PGw
len(df):  8
file:  H:\RS\Lexington\PNG\8312.PGw
len(df):  

len(df):  9
file:  H:\RS\Lexington\PNG\8449.PGw
len(df):  21
file:  H:\RS\Lexington\PNG\845.PGw
file:  H:\RS\Lexington\PNG\8450.PGw
len(df):  8
file:  H:\RS\Lexington\PNG\8451.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\8452.PGw
len(df):  15
file:  H:\RS\Lexington\PNG\8453.PGw
len(df):  16
file:  H:\RS\Lexington\PNG\8454.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\8455.PGw
len(df):  8
file:  H:\RS\Lexington\PNG\8456.PGw
len(df):  25
file:  H:\RS\Lexington\PNG\8457.PGw
len(df):  11
file:  H:\RS\Lexington\PNG\8458.PGw
len(df):  17
file:  H:\RS\Lexington\PNG\8459.PGw
len(df):  50
file:  H:\RS\Lexington\PNG\846.PGw
file:  H:\RS\Lexington\PNG\8460.PGw
len(df):  40
file:  H:\RS\Lexington\PNG\8461.PGw
len(df):  14
file:  H:\RS\Lexington\PNG\8462.PGw
len(df):  52
file:  H:\RS\Lexington\PNG\8463.PGw
len(df):  54
file:  H:\RS\Lexington\PNG\8464.PGw
len(df):  61
file:  H:\RS\Lexington\PNG\8465.PGw
len(df):  58
file:  H:\RS\Lexington\PNG\8466.PGw
len(df):  69
file:  H:\RS\Lexington\PNG\8467.PGw
len(

len(df):  4
file:  H:\RS\Lexington\PNG\8612.PGw
len(df):  13
file:  H:\RS\Lexington\PNG\8613.PGw
len(df):  3
file:  H:\RS\Lexington\PNG\8614.PGw
len(df):  6
file:  H:\RS\Lexington\PNG\8615.PGw
len(df):  6
file:  H:\RS\Lexington\PNG\8616.PGw
len(df):  8
file:  H:\RS\Lexington\PNG\8617.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\8618.PGw
len(df):  5
file:  H:\RS\Lexington\PNG\8619.PGw
len(df):  6
file:  H:\RS\Lexington\PNG\862.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\8620.PGw
len(df):  11
file:  H:\RS\Lexington\PNG\8621.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\8622.PGw
len(df):  93
file:  H:\RS\Lexington\PNG\8623.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\8624.PGw
len(df):  82
file:  H:\RS\Lexington\PNG\8625.PGw
len(df):  92
file:  H:\RS\Lexington\PNG\8626.PGw
len(df):  59
file:  H:\RS\Lexington\PNG\8627.PGw
len(df):  93
file:  H:\RS\Lexington\PNG\8628.PGw
len(df):  92
file:  H:\RS\Lexington\PNG\8629.PGw
len(df):  87
file:  H:\RS\Lexington\PNG\863.PGw
len(df):  8
file:  H:\RS\Lexingto

len(df):  6
file:  H:\RS\Lexington\PNG\8777.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\8778.PGw
len(df):  4
file:  H:\RS\Lexington\PNG\8779.PGw
len(df):  23
file:  H:\RS\Lexington\PNG\878.PGw
len(df):  4
file:  H:\RS\Lexington\PNG\8780.PGw
len(df):  14
file:  H:\RS\Lexington\PNG\8781.PGw
len(df):  8
file:  H:\RS\Lexington\PNG\8782.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\8783.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\8784.PGw
len(df):  14
file:  H:\RS\Lexington\PNG\8785.PGw
len(df):  19
file:  H:\RS\Lexington\PNG\8786.PGw
len(df):  27
file:  H:\RS\Lexington\PNG\8787.PGw
len(df):  56
file:  H:\RS\Lexington\PNG\8788.PGw
len(df):  86
file:  H:\RS\Lexington\PNG\8789.PGw
len(df):  22
file:  H:\RS\Lexington\PNG\879.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\8790.PGw
len(df):  62
file:  H:\RS\Lexington\PNG\8791.PGw
len(df):  34
file:  H:\RS\Lexington\PNG\8792.PGw
len(df):  3
file:  H:\RS\Lexington\PNG\8793.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\8794.PGw
len(df):  1
file:  H:\RS\Lexingt

file:  H:\RS\Lexington\PNG\918.PGw
len(df):  3
file:  H:\RS\Lexington\PNG\919.PGw
len(df):  8
file:  H:\RS\Lexington\PNG\92.PGw
len(df):  5
file:  H:\RS\Lexington\PNG\920.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\921.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\922.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\923.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\924.PGw
len(df):  4
file:  H:\RS\Lexington\PNG\925.PGw
len(df):  8
file:  H:\RS\Lexington\PNG\926.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\927.PGw
len(df):  4
file:  H:\RS\Lexington\PNG\928.PGw
len(df):  4
file:  H:\RS\Lexington\PNG\929.PGw
len(df):  3
file:  H:\RS\Lexington\PNG\93.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\930.PGw
len(df):  4
file:  H:\RS\Lexington\PNG\931.PGw
len(df):  7
file:  H:\RS\Lexington\PNG\932.PGw
len(df):  8
file:  H:\RS\Lexington\PNG\933.PGw
len(df):  2
file:  H:\RS\Lexington\PNG\934.PGw
len(df):  1
file:  H:\RS\Lexington\PNG\935.PGw
len(df):  3
file:  H:\RS\Lexington\PNG\936.PGw
len(df):  9
file:  H:\RS\Le

In [32]:
df = pd.read_csv(r'E:\Image_tweets\images201701\YOLO_label.csv')

In [33]:
df

,top,left,bottom,right,class_name,score,out_classes,image,area_pct
0,220,281,360,368,person,0.978434,0,1000005_816108782052929536-2.jpg,0.030825
1,251,88,359,126,person,0.899497,0,1000005_816108782052929536-2.jpg,0.010386
2,336,986,675,1200,person,0.995695,0,1000058_816108787107057664.jpg,0.089563
3,22,623,650,1163,person,0.961290,0,1000058_816108787107057664.jpg,0.418667
4,0,307,793,889,person,0.998556,0,1000088_816108790860877824.jpg,0.480756
5,385,664,795,952,person,0.995487,0,1000088_816108790860877824.jpg,0.123000
6,519,3,800,161,person,0.962127,0,1000088_816108790860877824.jpg,0.046248
7,0,203,630,952,person,0.998657,0,1000105_816108789631950848.jpg,0.582556
8,56,3,618,267,person,0.962569,0,1000105_816108789631950848.jpg,0.183170
9,0,27,572,995,person,0.999949,0,100015_819442976887894016.jpg,0.969475


In [53]:
ddd = df.groupby(['class_name']).agg({'class_name': 'count'})
#ddd.columns = ["_".join(x) for x in df.columns.ravel()]
print(type(ddd))

ddd.to_csv(r'D:\YOLO_labe.csv')


#df = df.sort_values(by=['count'])

 

<class 'pandas.core.frame.DataFrame'>


In [6]:
class_name.sort_values('count')

KeyError: 'count'

In [1]:
import numpy as np
from numba import vectorize

In [4]:

@vectorize(['float32(float32, float32)'], target='cuda')
def Add(a, b):
  return a + b

# Initialize arrays
N = 100000
A = np.ones(N, dtype=np.float32)
B = np.ones(A.shape, dtype=A.dtype)
C = np.empty_like(A, dtype=A.dtype)

# Add arrays on GPU
C = Add(A, B)

NvvmSupportError: libNVVM cannot be found. Do `conda install cudatoolkit`:
library nvvm not found